<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DEEPSEEK_AGENT_CREWAI_AOC_DEMO_JUNE2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jun 25 10:30:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   36C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade crewai langchain -q
!pip install --upgrade colab-env --quiet
!pip install --upgrade langchain_community -q
!pip install --upgrade faiss-cpu -q
!pip install --upgrade crewai-tools -q
!pip install 'crewai[tools]' -q
!pip install transformers -U -q

In [3]:
import warnings

warnings.filterwarnings("ignore", message="You seem to be using the pipelines sequentially on GPU")

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive


In [4]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import HuggingFaceHub

In [5]:
import colab_env
import os

In [6]:
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [7]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from litellm import completion  # Import litellm directly

load_dotenv()

# 1. Token Management (Assuming DeepSeek API key is in HUGGINGFACE_API_KEY environment variable)
access_token_write = os.getenv("HUGGINGFACE_API_KEY")

# 2. Initialize LLM (Using DeepSeek-V3)
model_name = "deepseek-ai/DeepSeek-V3"

# 3. Define Agents (Modify agent behavior to use litellm.completion)
def agent_run_with_litellm(self, task_description):
    """
    This function replaces the default agent run behavior to use litellm.completion.
    """
    messages = [{"role": "user", "content": task_description}]
    response = completion(model=model_name, messages=messages, api_key=access_token_write)
    return response.choices[0].message.content

# Monkey patch the agent's run method
Agent.run = agent_run_with_litellm

# 4. Define Agents (AIRLINE OPERATION CONTROL)
scheduler = Agent(
    role='Flight Scheduler',
    goal='Optimize flight schedules and manage aircraft assignments.',
    backstory="You are an experienced flight scheduler responsible for creating efficient flight plans and minimizing delays.",
    allow_delegation=False,
    verbose=True,
)

crew_dispatcher = Agent(
    role='Crew Dispatcher',
    goal='Assign crew members to flights based on qualifications and regulations.',
    backstory="You are a skilled crew dispatcher ensuring flights are staffed with qualified personnel while adhering to regulations.",
    allow_delegation=False,
    verbose=True,
)

disruption_handler = Agent(
    role='Disruption Handler',
    goal='Manage flight disruptions, re-routing aircraft and passengers as needed.',
    backstory="You are a resourceful disruption handler, adept at finding solutions to minimize the impact of unexpected events.",
    allow_delegation=False,
    verbose=True,
)

passenger_coordinator = Agent(
    role='Passenger Coordinator',
    goal='Assist passengers with travel arrangements, rebooking, and special needs.',
    backstory="You are a dedicated passenger coordinator, ensuring a smooth and positive travel experience for all passengers.",
    allow_delegation=False,
    verbose=True,
)

maintenance_coordinator = Agent(
    role='Maintenance Coordinator',
    goal='Schedule aircraft maintenance and manage repair resources.',
    backstory="You are a skilled maintenance coordinator, ensuring aircraft are airworthy and maintained according to regulations.",
    allow_delegation=False,
    verbose=True,
)

# 5. Define Tasks (AIRLINE OPERATION CONTROL)
schedule_optimization_task = Task(
    description="Optimize the flight schedule for the next 24 hours, considering aircraft availability, crew schedules, and expected passenger demand. Minimize delays and maximize aircraft utilization.",
    agent=scheduler,
    expected_output="An optimized flight schedule for the next 24 hours with aircraft and crew assignments.",
)

crew_assignment_task = Task(
    description="Assign crew members to the flights in the optimized schedule, ensuring all roles are filled by qualified personnel and regulatory requirements are met.",
    agent=crew_dispatcher,
    expected_output="A crew assignment plan for all flights in the schedule.",
)

disruption_management_task = Task(
    description="A severe weather event is causing delays at a major hub airport. Develop a plan to minimize disruption to passengers and operations, including re-routing aircraft and crew.",
    agent=disruption_handler,
    expected_output="A disruption management plan outlining actions to mitigate the impact of the weather event.",
)

passenger_assistance_task = Task(
    description="A flight is delayed, and passengers need to be rebooked on alternative flights or provided with accommodations. Coordinate with the scheduler and crew dispatcher to minimize passenger inconvenience.",
    agent=passenger_coordinator,
    expected_output="A passenger assistance plan for the delayed flight, including rebooking options and accommodation arrangements.",
)

maintenance_scheduling_task = Task(
    description="An aircraft requires routine maintenance. Schedule the maintenance during a non-operational window to minimize disruption to the flight schedule.",
    agent=maintenance_coordinator,
    expected_output="A maintenance schedule for the aircraft, ensuring minimal impact on flight operations.",
)

# 6. Create and Run the Crew
crew = Crew(
    agents=[scheduler, crew_dispatcher, disruption_handler, passenger_coordinator, maintenance_coordinator],
    tasks=[schedule_optimization_task, crew_assignment_task, disruption_management_task, passenger_assistance_task, maintenance_scheduling_task],
    process=Process.sequential
)

try:
    result = crew.kickoff()
    print("Crew's Output:")
    print(result)

except Exception as e:
    print(f"An error occurred during crew execution: {e}")

# Agent: Flight Scheduler
## Task: Optimize the flight schedule for the next 24 hours, considering aircraft availability, crew schedules, and expected passenger demand. Minimize delays and maximize aircraft utilization.


# Agent: Flight Scheduler
## Final Answer: 
**Optimized Flight Schedule for Next 24 Hours**

**Date: [Insert Current Date]**

**Aircraft Assignments:**
1. Aircraft A320-01
   - Availability: 08:00 - 22:00
   - Assigned to Flight AI-102 (Departure: 09:00, Arrival: 11:00, Destination: JFK)
   - Crew: Captain John Smith, First Officer Sarah Lee, Flight Attendant Team A

2. Aircraft B737-02
   - Availability: 07:30 - 23:00
   - Assigned to Flight AI-204 (Departure: 10:00, Arrival: 12:30, Destination: LAX)
   - Crew: Captain Mike Johnson, First Officer Emily Chen, Flight Attendant Team B

3. Aircraft A350-03
   - Availability: 06:00 - 20:00
   - Assigned to Flight AI-301 (Departure: 12:30, Arrival: 15:00, Destination: SFO)
   - Crew: Captain Lisa Wright, First Officer Davi